In [1]:
# Import some libraries

import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.datasets as datasets

import os
import pickle
import numpy as np

from skimage import io

from matplotlib import pyplot as plt
%matplotlib inline

import matplotlib as mpl
mpl.rcParams['axes.grid'] = False
mpl.rcParams['image.interpolation'] = 'nearest'
mpl.rcParams['figure.figsize'] = 15, 25

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 

df = pd.read_csv('/Users/nikhilvs/repos/nyu/birds-eye-view-predictor/artifacts/data/camera_data/annotation.csv')



In [16]:
df2 = df[df['category'] == 'car'].groupby(['scene', 'sample', 'category']).count()

df2.sort_values('action')

action  fl_x  fr_x  bl_x  br_x  fl_y  fr_y  bl_y  br_y  \
scene sample category                                                           
110   90     car            1     1     1     1     1     1     1     1     1   
      99     car            1     1     1     1     1     1     1     1     1   
      100    car            1     1     1     1     1     1     1     1     1   
      101    car            1     1     1     1     1     1     1     1     1   
      112    car            1     1     1     1     1     1     1     1     1   
...                       ...   ...   ...   ...   ...   ...   ...   ...   ...   
127   38     car           47    47    47    47    47    47    47    47    47   
      41     car           47    47    47    47    47    47    47    47    47   
      35     car           48    48    48    48    48    48    48    48    48   
      34     car           48    48    48    48    48    48    48    48    48   
      33     car           49    49    49    49    49    49    49    49    49   

                       category_id  action_id  new_scene  
scene sample category                                     
110   90     car                 1          1          1  
      99     car                 1          1          1  
      100    car                 1          1          1  
      101    car                 1          1          1  
      112    car                 1          1          1  
...                            ...        ...        ...  
127   38     car                47         47         47  
      41     car                47         47         47  
      35     car                48         48         48  
      34     car                48         48         48  
      33     car                49         49         49  

[3503 rows x 12 columns]

In [8]:
df.head()

,scene,sample,action,category,fl_x,fr_x,bl_x,br_x,fl_y,fr_y,bl_y,br_y,category_id,action_id,new_scene
0,111,0,object_action_driving_straight_forward,car,-14.270185,-14.286638,-9.696476,-9.712929,19.254325,21.132901,19.294995,21.173571,2,1,111
1,111,0,object_action_parked,car,-5.902891,-5.657694,-1.628863,-1.383666,29.164061,31.026680,28.602180,30.464799,2,0,111
2,111,0,object_action_walking,pedestrian,37.068518,36.283792,37.300167,36.515442,-20.233128,-20.032588,-19.327417,-19.126877,3,2,111
3,111,0,object_action_driving_straight_forward,car,1.972478,1.988930,-2.744222,-2.727770,8.132214,6.253638,8.090272,6.211696,2,1,111
4,111,0,object_action_driving_straight_forward,car,39.051316,39.067769,34.334616,34.351068,5.287296,3.408720,5.245354,3.366778,2,1,111


In [19]:
import torch
from torch import nn
from torchvision import models

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = models.resnet18()
model.fc = nn.Linear(512, 500)
criterion = nn.MSELoss()
model = model.to(device)


batch_size = 32
workers = 2



In [25]:
from torchsummary import summary

summary(model, (3, 256, 306))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 153]           9,408
       BatchNorm2d-2         [-1, 64, 128, 153]             128
              ReLU-3         [-1, 64, 128, 153]               0
         MaxPool2d-4           [-1, 64, 64, 77]               0
            Conv2d-5           [-1, 64, 64, 77]          36,864
       BatchNorm2d-6           [-1, 64, 64, 77]             128
              ReLU-7           [-1, 64, 64, 77]               0
            Conv2d-8           [-1, 64, 64, 77]          36,864
       BatchNorm2d-9           [-1, 64, 64, 77]             128
             ReLU-10           [-1, 64, 64, 77]               0
       BasicBlock-11           [-1, 64, 64, 77]               0
           Conv2d-12           [-1, 64, 64, 77]          36,864
      BatchNorm2d-13           [-1, 64, 64, 77]             128
             ReLU-14           [-1, 64,

In [28]:
x = torch.randn((1, 3, 256, 306))
model(x).shape

torch.Size([1, 500])